In [2]:
import torch
from torch import nn


class Conv2dSame(nn.Module):
    """ Applies a 2D convolution  over an input signal composed of several input
        planes and output size is the same as input size.
        Args:
            in_channels (int): Number of channels in the input image
            out_channels (int): Number of channels produced by the convolution
            kernel_size (only tuple): Size of the convolving kernel
            stride (only int): Stride of the convolution. Default: 1
            padding_layer: type of layer for padding. Default: nn.ZeroPad2d
            dilation (only int): Spacing between kernel elements. Default: 1
            bias (bool): If ``True``, adds a learnable bias to the output. Default: ``True``

        Shape:
            - Input: :math:`(N, C_{in}, H_{in}, W_{in})`
            - Output: :math:`(N, C_{out}, H_{out}, W_{out})` where
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding_layer=nn.ZeroPad2d, dilation=1, bias=True):
        super().__init__()

        kernel_h, kernel_w = kernel_size

        k = dilation * (kernel_w - 1) - stride + 2
        pr = k // 2
        pl = pr - 1 if k % 2 == 0 else pr

        k = dilation * (kernel_h - 1) - stride + 1 + 1
        pb = k // 2
        pt = pb - 1 if k % 2 == 0 else pb
        self.pad_same = padding_layer((pl, pr, pb, pt))
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride, dilation=dilation, bias=bias)

    def forward(self, x):
        x = self.pad_same(x)
        x = self.conv(x)
        return x

In [3]:
c = Conv2dSame(1,64,(4,4),1)  #64

c1 = Conv2dSame(64,100,(4,4),1)  #64

h = nn.Conv2d(100,127,4,2,1) #32

c2 = Conv2dSame(127,512,(4,4),1) #32

h2 = nn.Conv2d(512,1,4,2,1) #16

y = h2(c2(h(c1(c(torch.rand((64,1,64,64))))))) #16


y.shape

# c2(c(torch.rand((64,1,64,64)))).shape
# c3 = Conv2dSame(127,387,(4,4),1)
# y = c3(c2(c(torch.rand((64,1,64,64)))))

# print(y.shape)

torch.Size([64, 1, 16, 16])

In [ ]:
import torch.nn as nn
import torch
import math

class Conv2dSame(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(Conv2dSame, self).__init__()
        self.F = kernel_size
        self.S = stride
        self.D = dilation
        self.layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, dilation=dilation)

    def forward(self, x_in):
        N, C, H, W = x_in.shape
        H2 = math.ceil(H / self.S)
        W2 = math.ceil(W / self.S)
        Pr = (H2 - 1) * self.S + (self.F - 1) * self.D + 1 - H
        Pc = (W2 - 1) * self.S + (self.F - 1) * self.D + 1 - W
        x_pad = nn.ZeroPad2d((Pr//2, Pr - Pr//2, Pc//2, Pc - Pc//2))(x_in)
        x_out = self.layer(x_pad)
        return x_out

In [ ]:
# x.shape

b = Conv2dSame(1,64,4,1)
print(b(torch.rand((64,1,64,64))).shape)

In [14]:
from numpy.random import randint

ngpu =1
noDlayers = 10
activationF = [randint(0, 3) for x in range(noDlayers-1)]
opChannels = [randint(64, 513) for i in range(noDlayers-1)]
same_conv2d_pos = {5: [0, 0, 0, 0, 0],
                       6: [0, 1, 0, 0, 0, 0],
                       7: [0, 1, 0, 1, 0, 0, 0],
                       8: [0, 1, 0, 1, 0, 1, 0, 0],
                       9: [0, 1, 0, 1, 0, 1, 0, 1, 0],
                       10: [0, 1, 1, 0, 1, 0, 1, 0, 1, 0]}
nc =1
ndf =64
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

class Discriminator(nn.Module):
    def __init__(self, ngpu, opChannels, activationF,same_conv2d_pos):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.activationF = activationF
        self.opChannels = opChannels


        ip = nc
        op = ndf
        i = 0
        model = []
#         op = self.opChannels[i]
        while i < noDlayers-1:
            
            if same_conv2d_pos[noDlayers][i] == 1:
                # Add a 6th same padding layer !
                model += [Conv2dSame(ip, op, (4, 4), 1)]
            else:
                model += [nn.Conv2d(ip, op, 4, 2, 1)]
            
            if i > 0:
                model += [nn.BatchNorm2d(op)]

            if self.activationF[i] == 0:
                model += [nn.LeakyReLU(0.2)]
            elif self.activationF[i] == 1:
                model += [nn.ReLU(inplace=True)]
            elif self.activationF[i] == 2:
                model += [nn.ELU(inplace=True)]


            ip = op
            op = self.opChannels[i]

            i = i+1
        model += [
            nn.Conv2d(ip, 1, 4, 1),
            nn.Sigmoid()]
        self.main = torch.nn.Sequential(*model)

    def forward(self, input):
        return self.main(input)
    

    
netD = Discriminator(1,opChannels,activationF,same_conv2d_pos).to(device)

In [15]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
dataroot = "C:/Users/RACHIT/Desktop/coegan-master/mnist_png/training"
dataset = dset.ImageFolder(root=dataroot,
                               transform=transforms.Compose([
                                   transforms.Grayscale(1),
                                   transforms.Resize(64),
                                   transforms.CenterCrop(64),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                             shuffle=True, num_workers=1)


def weights_init(m):
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

netD.zero_grad()
data = next(iter(dataloader))
real_cpu = data[0].to(device)
print(real_cpu.shape)

netD.apply(weights_init)




torch.Size([64, 1, 64, 64])


Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2dSame(
      (pad_same): ZeroPad2d(padding=(1, 2, 2, 1), value=0.0)
      (conv): Conv2d(64, 331, kernel_size=(4, 4), stride=(1, 1))
    )
    (3): BatchNorm2d(331, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace)
    (5): Conv2dSame(
      (pad_same): ZeroPad2d(padding=(1, 2, 2, 1), value=0.0)
      (conv): Conv2d(331, 226, kernel_size=(4, 4), stride=(1, 1))
    )
    (6): BatchNorm2d(226, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ELU(alpha=1.0, inplace)
    (8): Conv2d(226, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2dSame(
      (pad_same): ZeroPad2d(padding=(1, 2, 2, 1), value=0.0)
      (conv): Conv2d(64, 233, kernel_size=(4, 4), s

In [16]:
output = netD(real_cpu)
print(output.shape)
# output .view(-1)


torch.Size([64, 1, 1, 1])


In [13]:

print(output.view(-1).shape)

torch.Size([64])


In [ ]:
import torch
print(torch.__version__)